In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import numpy as np

sys.path.insert(0, '../../../batchflow')
sys.path.insert(0, '../../')

from seismiqb import SeismicDataset, HorizonMetrics, SeismicSampler
from batchflow import Pipeline, B

In [ ]:
GEOMETRY_PATH = 'cube.blosc'
HORIZONS_PATH = 'horizons/*'

dataset = SeismicDataset({GEOMETRY_PATH: {'horizons': HORIZONS_PATH}}, interpolate=True)
print(dataset)

In [ ]:
field = dataset[0]
horizon = field.horizons[0]

### Inline/crossline-oriented slices sampling

In [ ]:
CROP_SHAPE = (1, 128, 256)

sampler = SeismicSampler(labels=dataset.labels, crop_shape=CROP_SHAPE)

sampler.show_sampled()

In [ ]:
def make_horizons_predictions(batch, q=0.95):
    predictions = []
    for image in batch.images:
        val = np.quantile(image, q)
        prediction = np.zeros_like(image)
        prediction[image > val] = image[image > val] / np.nanmax(image)
        predictions.append(prediction)
    batch.predictions = np.array(predictions, dtype='object')

In [ ]:
BATCH_SIZE = 32

template = (
    Pipeline()
    .make_locations(generator=sampler, batch_size=BATCH_SIZE)
    .load_cubes(dst='images')
    .create_masks(dst='masks')
    .make_horizons_predictions(B())
)
pipeline = template << dataset

In [ ]:
batch = pipeline.next_batch()
batch.plot()

In [ ]:
batch.plot(dilate=[None, 1, [None, 1], None, 1])

### Depth-oriented slices sampling

In [ ]:
CROP_SHAPE = (256, 256, 1)

sampler = SeismicSampler(labels=dataset.labels, crop_shape=CROP_SHAPE, threshold=0., mode='horizon', depth_shift=False)

sampler.show_sampled()

In [ ]:
def make_facies_masks(batch, threshold=0.5):
    masks = []

    for image in batch.images:
        val = np.quantile(image, threshold)
        mask = np.zeros_like(image)
        mask = image > val
        masks.append(mask)

    batch.masks = np.array(masks)
    
def make_facies_predictions(batch, threshold=0.6):
    predictions = []

    for image in batch.images:
        prediction = np.zeros_like(image)
        val = np.quantile(image, threshold)
        mask = image > val
        min_, max_ = np.nanmin(image[mask]), np.nanmax(image[mask])
        prediction[mask] = (image[mask] - min_) / (max_ - min_)
        predictions.append(prediction)
        
    batch.predictions = np.array(predictions)

In [ ]:
BATCH_SIZE = 32

template = (
    Pipeline()
    .make_locations(generator=sampler, batch_size=BATCH_SIZE)
    .compute_label_attribute(src='horizons:*/amplitudes', dst='images', fill_value=0)
    .make_facies_masks(B())
    .make_facies_predictions(B())
)
pipeline = template << dataset

In [ ]:
batch = pipeline.next_batch()
plotter = batch.plot()

In [ ]:
batch.plot_roll(n=3)